## Jupyter Notebook GPU Detection Test
Run the code below, do not modify it.

In [1]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.1.0
Default GPU Device: /device:GPU:0


In [2]:
import numpy
import csv

filename = "jokes_plain_tweet_data.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# strip unwanted characters
raw_text = raw_text.replace('"', '')
raw_text = raw_text.replace('\n', '')
raw_text = raw_text.replace('#', '')

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

n_chars = len(raw_text)
n_vocab = len(chars)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 140

dataX = []
dataY = []

# creates a sequence of 100 characters, output contains the corrosponding character that follows.
# window of 'seq_length' that increments across each character.

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:(i + seq_length)]
    seq_out = raw_text[i + seq_length]
    
    #append to training data
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
    
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
Total Patterns:  133196


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# reshape X to be [samples, time steps, features]
#X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)


# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(128))
model.add(Dropout(0.25))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# define the checkpoint
filepath="weights\weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


history = model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Using TensorFlow backend.


Epoch 1/10
133196/133196 [==============================] - 189s 1ms/step - loss: 2.8842

Epoch 00001: loss improved from inf to 2.88419, saving model to weights\weights-improvement-01-2.8842.hdf5


OSError: Unable to create file (unable to open file: name = 'weights\weights-improvement-01-2.8842.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)